In [1]:
import pandas as pd
import os
import prepare_data.mind as mind
import tensorflow as tf

In [2]:
epochs = 5
seed = 40
MIND_type = 'demo'
# tmpdir = TemporaryDirectory()
# data_path = tmpdir.name
data_path = os.path.join('./data', 'mind-{}'.format(MIND_type))
print(data_path)

./data\mind-demo


In [3]:
train_news_file = os.path.join(data_path, 'train', r'news.tsv')
train_behaviors_file = os.path.join(data_path, 'train', r'behaviors.tsv')
valid_news_file = os.path.join(data_path, 'valid', r'news.tsv')
valid_behaviors_file = os.path.join(data_path, 'valid', r'behaviors.tsv')
wordEmb_file = os.path.join(data_path, "utils", "embedding.npy")
userDict_file = os.path.join(data_path, "utils", "uid2index.pkl")
wordDict_file = os.path.join(data_path, "utils", "word_dict.pkl")
yaml_file = os.path.join(data_path, "utils", r'lstur.yaml')

In [4]:
new_headers = ['news_id',
               'category',
               'subcategory',
               'title',
               'abstract',
               'url',
               'title_entities',
               'abstract_entities']
news_data = pd.read_csv(train_news_file, sep='\t', names=new_headers)
news_data.head(5)

,news_id,category,subcategory,title,abstract,url,title_entities,abstract_entities
0,N3112,lifestyle,lifestyleroyals,"The Brands Queen Elizabeth, Prince Charles, an...","Shop the notebooks, jackets, and more that the...",https://www.msn.com/en-us/lifestyle/lifestyler...,"[{""Label"": ""Prince Philip, Duke of Edinburgh"",...",[]
1,N10399,news,newsworld,The Cost of Trump's Aid Freeze in the Trenches...,Lt. Ivan Molchanets peeked over a parapet of s...,https://www.msn.com/en-us/news/world/the-cost-...,[],"[{""Label"": ""Ukraine"", ""Type"": ""G"", ""WikidataId..."
2,N12103,health,voices,I Was An NBA Wife. Here's How It Affected My M...,"I felt like I was a fraud, and being an NBA wi...",https://www.msn.com/en-us/health/voices/i-was-...,[],"[{""Label"": ""National Basketball Association"", ..."
3,N20460,health,medical,"How to Get Rid of Skin Tags, According to a De...","They seem harmless, but there's a very good re...",https://www.msn.com/en-us/health/medical/how-t...,"[{""Label"": ""Skin tag"", ""Type"": ""C"", ""WikidataI...","[{""Label"": ""Skin tag"", ""Type"": ""C"", ""WikidataI..."
4,N5409,weather,weathertopstories,It's been Orlando's hottest October ever so fa...,There won't be a chill down to your bones this...,https://www.msn.com/en-us/weather/weathertopst...,"[{""Label"": ""Orlando, Florida"", ""Type"": ""G"", ""W...","[{""Label"": ""Orlando, Florida"", ""Type"": ""G"", ""W..."


In [5]:
behaviors_header = ['impression_id',
                    'user_id',
                    'time',
                    'history',
                    'impressions']

behaviors_data = pd.read_csv(train_behaviors_file, sep='\t', names=behaviors_header)

In [6]:
behaviors_data

,impression_id,user_id,time,history,impressions
0,1,U82271,11/11/2019 3:28:58 PM,N3130 N11621 N12917 N4574 N12140 N9748,N13390-0 N7180-0 N20785-0 N6937-0 N15776-0 N25...
1,2,U84185,11/12/2019 10:36:47 AM,N27209 N11723 N4617 N12320 N11333 N24461 N2211...,N13089-0 N18101-0 N1248-0 N26273-0 N12770-1 N1...
2,3,U11552,11/11/2019 1:03:52 PM,N2139,N18390-0 N10537-0 N23967-1
3,4,U68381,11/11/2019 6:44:05 AM,N27420 N11621 N25416 N25457 N5124 N11751 N1175...,N15660-0 N18609-0 N2831-0 N5677-0 N19010-0 N15...
4,5,U52303,11/12/2019 3:11:52 AM,N1332 N12667,N15645-0 N7911-1
...,...,...,...,...,...
22029,22030,U69508,11/12/2019 2:50:55 AM,N19377 N5490 N19086 N16129 N2746 N21659 N17510...,N16297-0 N15645-0 N20630-0 N10602-0 N27294-0 N...
22030,22031,U79085,11/14/2019 9:52:15 AM,N27591 N21778 N17297 N9764 N13579 N22605 N2585...,N27355-0 N9809-0 N20882-0 N8787-0 N25926-0 N38...
22031,22032,U46989,11/14/2019 8:36:23 AM,N4202 N23413 N3617 N19239 N11061 N22055 N1839 ...,N13316-0 N23592-0 N10240-0 N1994-0 N26998-0 N1...
22032,22033,U49050,11/14/2019 11:58:12 AM,N17580 N12014 N3076 N10624 N27610 N6385 N4617 ...,N23982-0 N17424-0 N17157-0 N4324-0 N22216-0 N1...


In [7]:
behaviors_data[behaviors_data.user_id == 'U79085']['impressions'].values.tolist()[1].split(' ')

['N5821-0',
 'N12252-0',
 'N13862-0',
 'N4460-0',
 'N9645-0',
 'N2127-0',
 'N16787-0',
 'N10767-0',
 'N26803-0',
 'N12881-0',
 'N10729-0',
 'N11011-0',
 'N5558-0',
 'N2177-0',
 'N2974-0',
 'N15827-0',
 'N5309-1',
 'N16385-0',
 'N15959-0',
 'N21206-0',
 'N20702-0',
 'N13390-0',
 'N6452-0',
 'N14777-0',
 'N2579-0',
 'N1968-0',
 'N21807-0',
 'N9950-0',
 'N19430-0',
 'N4566-0',
 'N24674-0',
 'N3867-0',
 'N8078-0',
 'N19375-0',
 'N25419-0',
 'N10074-0',
 'N23029-0',
 'N16817-0',
 'N25163-0',
 'N27077-0',
 'N27428-0',
 'N18101-0',
 'N12709-0',
 'N21000-0',
 'N12167-0']

In [8]:
behaviors_data[behaviors_data.user_id == 'U82271']['history'].values.tolist()[0].split(' ')

['N3130', 'N11621', 'N12917', 'N4574', 'N12140', 'N9748']

In [13]:
header = ['user_id', 'song_id', 'gmt_create', 'action_type', 'Ds']

data = pd.read_csv('./data/aliMusic/p2_mars_tianchi_user_actions.csv', nrows=100, names=header)

In [14]:
data

,user_id,song_id,gmt_create,action_type,Ds
0,7063b3d0c075a4d276c5f06f4327cf4a,effb071415be51f11e845884e67c0f8c,1426406400,1,20150315
1,0db66c0dd3993fd3504bb98c3beb15b3,f87ff481d85d2f95335ab602f38a7655,1426417200,1,20150315
2,f8c065dc140ec74c6e44144164e618e3,8a27d9a6c59628c991c154e8d93f412e,1426406400,2,20150315
3,2c6082cf0d68e244f2a10325e8d1b85b,ecea5fe33e6817d09c395f2910479728,1426417200,1,20150315
4,2c6082cf0d68e244f2a10325e8d1b85b,31a3d0420d89c9b121bb55dbdbbeda6b,1426417200,1,20150315
...,...,...,...,...,...
95,96ed0a5d45ed56deddfbf01e61dd6408,2ddd2aa8a3d4438e70feb2784ba7a2fd,1426420800,2,20150315
96,a7e41d6e522a6983b9c2b51bff8e73e7,0ab5e6c4f3e03b9959c2459fb53aaa0e,1426374000,1,20150315
97,70c6d61e16f2b2ef9c1e1651363060f2,b7cd9966e6dcd507e7e13ef58fbef3cc,1426399200,1,20150315
98,62de91162e452694d4ac994981747544,0136e4c353c5ea29ffea1e7e912077ec,1426410000,1,20150315


In [16]:
song_header = ['song_id',
               'artist_id',
               'publish_time',
               'song_init_plays',
               'language',
               'gender']
data = pd.read_csv('./data/aliMusic/p2_mars_tianchi_songs.csv', nrows=100, names=song_header)
data

,song_id,artist_id,publish_time,song_init_plays,language,gender
0,c81f89cf7edd24930641afa2e411b09c,03c6699ea836decbc5c8fc2dbae7bd3b,20150325,0,100,1
1,c0d7130777c1f1c417e78646946ed909,03c6699ea836decbc5c8fc2dbae7bd3b,20150325,0,100,1
2,200c9131cf929bab418d380356be5f42,03c6699ea836decbc5c8fc2dbae7bd3b,20150325,0,100,1
3,78fedfdf13fc820e363e39986ff91e94,03c6699ea836decbc5c8fc2dbae7bd3b,20110910,1717,1,1
4,95b99faf432d33772d63f828bf2d0921,03c6699ea836decbc5c8fc2dbae7bd3b,20110910,434,1,1
...,...,...,...,...,...,...
95,043846191b18e10f6e5c5e8e25380a8b,03c6699ea836decbc5c8fc2dbae7bd3b,20110910,415,1,1
96,8241a6fa578787185861d7440cc77d79,03c6699ea836decbc5c8fc2dbae7bd3b,20130817,762,1,1
97,b79bc880e189a3bfc1880f898e83210a,03c6699ea836decbc5c8fc2dbae7bd3b,20160207,6761,0,1
98,414f13b802ed56ccbe057cc591af5061,03c6699ea836decbc5c8fc2dbae7bd3b,20130817,3145,1,1
